## Part 1. Get metadata

In [464]:
import re
import sys
import pandas as pd
import urllib2 
from bs4 import BeautifulSoup 
import json

In [465]:
import csv
import requests 

In [466]:
from fake_useragent import UserAgent
import requests
from time import sleep

In [467]:
ua = UserAgent()
# print(ua.chrome)
header = {'User-Agent':str(ua.chrome)}
print(header)

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.90 Safari/537.36'}


In [468]:
from xml.dom import minidom

In [12]:
# Test_file = open('C:/Users/mariia.petryk/Downloads/736422b4-4ec4-4602-9871-ec0b9565eb23.xml','r')
# xmldoc = minidom.parse(Test_file)

In [469]:
f = open('C:/Users/mariia.petryk/Downloads/558f6811-0010-4cfb-86be-877d7ea9cd88.xml',"r")
data = f.readlines()

In [470]:
bs= BeautifulSoup(str(data))

In [489]:
j=0
k=0
batch_list=list()
auth=str("")
df=pd.DataFrame(columns=['Author', 'Title','Pub_type', 'Journal', 'Year_publish', 'Doi','Abstract','Keywords','Url_link'])
count=len(bs.findAll("rec"))
for i in range(0,count):
#     print i
    jour=bs.findAll("jtl")[i].get_text()
    df['Journal'].loc[i] =jour
    year=bs.findAll("dt")[i].get_text()
    df['Year_publish'].loc[i] =year[:-4].strip()
    try:
        doi=bs.findAll("ui", type="doi")[i].get_text()
        df['Doi'].loc[i] =doi
    except:
        df['Doi'].loc[i] =''
    title=bs.findAll("atl")[i].get_text()
    df['Title'].loc[i] =title
    auth=bs.findAll("aug")[i].get_text().strip('\\n')
    df['Author'].loc[i] =auth[4:].encode("utf-8").strip('\n')
#     print auth
    try:
        subj=bs.findAll("su")[i].get_text().strip('\\n')
        df['Keywords'].loc[i] =subj.strip('\\n')
    except:
        df['Keywords'].loc[i] =''
    absrt=bs.findAll("ab")[i].get_text().strip()
    df['Abstract'].loc[i] =absrt.encode("utf-8")
    url=bs.findAll("url")[i].get_text()
    df['Url_link'].loc[i] =url
#     print url

In [490]:
# Append columns into one dataframe
df = pd.concat([df['Author'],  df['Title'],df['Pub_type'],df['Journal'],df['Year_publish'],df['Doi'],df['Abstract'],df['Keywords'],df['Url_link']], axis=1)

In [491]:
df.head()

,Author,Title,Pub_type,Journal,Year_publish,Doi,Abstract,Keywords,Url_link
0,"Geuens, Stijn\n', ' Coussem...",A Framework for Configuring Collaborative Filt...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.07.005,This study proposes a decision support framewo...,Optimization Techniques; Programming Models; D...,http://search.ebscohost.com/login.aspx?direct=...
1,"Yang, Xinan\n', ' Strauss, ...",An Approximate Dynamic Programming Approach to...,NaN,European Journal of Operational Research,2017,http://dx.doi.org/10.1016/j.ejor.2017.06.034,We propose a new method of controlling demand ...,Retail and Wholesale Trade; e-Commerce,http://search.ebscohost.com/login.aspx?direct=...
2,"Yang, Huixiao\n', ' Luo, Ji...",Supplier Encroachment under Nonlinear Pricing ...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.027,We explore the impact of nonlinear pricing (NP...,Optimization Techniques; Programming Models; D...,http://search.ebscohost.com/login.aspx?direct=...
3,"Gharehgozli, Amir\n', ' Zae...",Stacking Outbound Barge Containers in an Autom...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.040,"In this paper, we study the stacking problem o...","Production; Cost; Capital; Capital, Total Fact...",http://search.ebscohost.com/login.aspx?direct=...
4,"Nicolas, Lenoble\n', ' Yann...",Order Batching in an Automated Warehouse with ...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.037,The vertical lift module is an automated stora...,Transactional Relationships; Contracts and Rep...,http://search.ebscohost.com/login.aspx?direct=...


In [492]:
df.to_csv('List_EJOR_ecommerce.csv')

## Next: (1) code download pdf - not working yet

## Part 2. Download papers pdf
## Note: papers may duplicate within different keywords, so filter unique paper links

In [602]:
f = pd.read_csv("C:\Users\mariia.petryk\List_EJOR_ecommercev03.csv")
# data = f.readlines()

In [603]:
len(f)

6

In [495]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.support import expected_conditions
# from selenium.webdriver.common.by import By

In [496]:
# via Selenium
# not good for Elsevier database (e.g. EJOR didn't fit)
# j=5


for j in range(0,len(f['Url_link'])):
    
    print j
    options = webdriver.ChromeOptions()
    download_folder = "C:\\Users\\mariia.petryk"    

    profile = {"plugins.plugins_list": [{"enabled": False,
                                         "name": "Chrome PDF Viewer"}],
               "download.default_directory": download_folder,
               "download.extensions_to_open": "applications/pdf"
              }

    options.add_experimental_option("prefs", profile)
    # set driver
    driver = webdriver.Chrome('C:\Users\mariia.petryk\Downloads\chromedriver_win32\chromedriver.exe', chrome_options=options)  # Optional argument, if not specified will search path.

    
    link=f['Url_link'].iloc[j]
    #put here the adress of your page
    driver.get(link)
    # sid and vid for pdf link
    s=driver.find_elements_by_xpath("//*[@id='__sid']")
    sid =s[0].get_attribute("value")
    v=driver.find_elements_by_xpath("//*[@id='__vid']")
    vid =v[0].get_attribute("value")
    #Getting local session of Chrome
    url = driver.command_executor._url      
    session_id = driver.session_id 

    ll='http://web.b.ebscohost.com/ehost/pdfviewer/pdfviewer?'+'vid='+vid+'&sid='+sid

    print("Downloading file from link: {}".format(ll))
    driver.get(ll)

    print("Status: Download Complete.")
    # print("Folder: {}".format(download_folder))
    # sleep(10)


    time.sleep(5)
    filename2=((' '.join(f['Author'].iloc[j].split()).replace(", ", "_").replace(",", "_"))+"_"+str(f['Year_publish'].iloc[j])+".pdf").lower()
    print filename2
    for filename in os.walk('C:\\Users\\mariia.petryk'):
        for fn in filename[2]:
            try:
                if fn.startswith("ContentServer"):
                    os.rename(fn, filename2)
            except:
                pass
    # time.sleep(5)
    wait = WebDriverWait(driver, 5)
    driver.close()

0
Status: Download Complete.
boysen_nils_briskorn_dirk_emde_simon_2017.pdf
1
Status: Download Complete.
lvarez-miranda_eduardo_luipersbeck_martin_sinnl_markus_2018.pdf
2
Status: Download Complete.
germs_remco_van foreest_nicky d._kilic_onur a._2016.pdf
3
Status: Download Complete.
avinadav_tal_chernonog_tatyana_perlman_yael__2015.pdf


In [320]:
# link=f['Url_link'].iloc[3]
# r = requests.get(link, stream = True, headers=header)

In [296]:
# text=r.text
# soup = BeautifulSoup(text, "lxml")
# gg=soup.findAll("script", {"class": "rs_skip"})

In [611]:
# via Selenium
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

j=6

#Getting local session of Chrome
driver=webdriver.Chrome('C:\Users\mariia.petryk\Downloads\chromedriver_win32\chromedriver.exe')
link=f['Url_link'].iloc[j]
#put here the adress of your page
driver.get(link)
s=driver.find_elements_by_xpath("//*[@id='__sid']")
sid =s[0].get_attribute("value")
v=driver.find_elements_by_xpath("//*[@id='__vid']")
vid =v[0].get_attribute("value")
url = driver.command_executor._url      
session_id = driver.session_id 
ll='http://web.b.ebscohost.com/ehost/pdfviewer/pdfviewer?'+'vid='+vid+'&sid='+sid
options = webdriver.ChromeOptions()
download_folder = "C:/Users/mariia.petryk"    
profile = {"plugins.plugins_list": [{"enabled": False,
                                     "name": "Chrome PDF Viewer"}],
           "download.default_directory": download_folder,
           "download.extensions_to_open": "",
           "download.directory_upgrade": True
          }
options.add_experimental_option("prefs", profile)
print("Downloading file from link: {}".format(ll))
filename=((' '.join(f['Author'].iloc[j].split()).replace(", ", "_").replace(",", "_"))+"_"+str(f['Year_publish'].iloc[j])).lower()
print("Status: Download Complete.")
print("Folder: {}".format(download_folder))
print filename

IndexError: single positional indexer is out-of-bounds

## Part 3. EJOR (Elsevier)
## Next: +(1) get list of keywords from Elsevier
## (2) download papers from Elsevier for EJOR

In [534]:
f = pd.read_csv("C:\Users\mariia.petryk\List_EJOR_ecommerce.csv")
# data = f.readlines()

In [535]:
len(f)

56

In [585]:
kw=pd.DataFrame(columns=['Keywords'])
klist=pd.DataFrame(columns=['Keywords'])

In [589]:
i=0
for i in range(0,len(f)):
    print i
    driver=webdriver.Chrome('C:\Users\mariia.petryk\Downloads\chromedriver_win32\chromedriver.exe')
    link=f['Doi'].iloc[i]
    #put here the adress of your page
    driver.get(link)
    time.sleep(2)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, "lxml")
    gg=soup.findAll("div", {"class": "keyword"})
    for k in range(0,len(gg)):
        text=gg[k].get_text()
        kw['Keywords'].loc[k]=text
    keylist=",".join(kw['Keywords'][0:k])
    klist['Keywords'].loc[i]=keylist
    print keylist
    kw=pd.DataFrame(columns=['Keywords'])
    sleep(2)
    driver.close()

0
E-commerce,OR in marketing,Recommendation systems,Collaborative filtering
1
E-commerce,Revenue management
2
E-commerce,Supplier encroachment,Bargaining power,Nonlinear price contracts
3
OR in Maritime Industry,Outbound barge containers,Stacking operations,Heuristic
4
Logistics,Automated warehouses,Batching optimization,Vertical lift module,Order picking,AS/RSAutomated Storage and Retrieval Systems,Automated Storage and Retrieval Systems,VLMVertical Lift Module,Vertical Lift Module,AVS/RSAutonomous Vehicle Storage and Retrieval Systems,Autonomous Vehicle Storage and Retrieval Systems,SBS/RSShuttle Based Storage and Retrieval Systems,Shuttle Based Storage and Retrieval Systems,SASimulated Annealing,Simulated Annealing,MFVMean Fitness Value,Mean Fitness Value,EREfficiency Rate,Efficiency Rate,IRImprovement Rate
5
Supply chain management,Inventory,Behavioral operations
6
Supply chain management,Perishables,Inventory
7
Supply chain management,Forecast sharing,Inventory commitments/realloc

In [591]:
df['Keywords'] = klist['Keywords']

In [599]:
for i in range(0,len(df['Author'])):
    df['Author'].loc[i]=df['Author'].loc[i].encode('ascii','ignore')
    df['Author'].loc[i]=df['Author'].loc[i].replace('\n','')
    df['Keywords'].loc[i]=df['Keywords'].loc[i].encode('ascii','ignore')
    df['Keywords'].loc[i]=df['Keywords'].loc[i].replace('\n','')

In [600]:
df.head()

,Author,Title,Pub_type,Journal,Year_publish,Doi,Abstract,Keywords,Url_link
0,"Geuens, Stijn\n', ' Coussem...",A Framework for Configuring Collaborative Filt...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.07.005,This study proposes a decision support framewo...,"E-commerce,OR in marketing,Recommendation syst...",http://search.ebscohost.com/login.aspx?direct=...
1,"Yang, Xinan\n', ' Strauss, ...",An Approximate Dynamic Programming Approach to...,NaN,European Journal of Operational Research,2017,http://dx.doi.org/10.1016/j.ejor.2017.06.034,We propose a new method of controlling demand ...,"E-commerce,Revenue management",http://search.ebscohost.com/login.aspx?direct=...
2,"Yang, Huixiao\n', ' Luo, Ji...",Supplier Encroachment under Nonlinear Pricing ...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.027,We explore the impact of nonlinear pricing (NP...,"E-commerce,Supplier encroachment,Bargaining po...",http://search.ebscohost.com/login.aspx?direct=...
3,"Gharehgozli, Amir\n', ' Zae...",Stacking Outbound Barge Containers in an Autom...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.040,"In this paper, we study the stacking problem o...","OR in Maritime Industry,Outbound barge contain...",http://search.ebscohost.com/login.aspx?direct=...
4,"Nicolas, Lenoble\n', ' Yann...",Order Batching in an Automated Warehouse with ...,NaN,European Journal of Operational Research,2018,http://dx.doi.org/10.1016/j.ejor.2017.12.037,The vertical lift module is an automated stora...,"Logistics,Automated warehouses,Batching optimi...",http://search.ebscohost.com/login.aspx?direct=...


In [601]:
df.to_csv('List_EJOR_ecommerce2.csv')

In [590]:
klist['Keywords']

0     E-commerce,OR in marketing,Recommendation syst...
1                         E-commerce,Revenue management
2     E-commerce,Supplier encroachment,Bargaining po...
3     OR in Maritime Industry,Outbound barge contain...
4     Logistics,Automated warehouses,Batching optimi...
5     Supply chain management,Inventory,Behavioral o...
6         Supply chain management,Perishables,Inventory
7     Supply chain management,Forecast sharing,Inven...
8     Retailing,Traditional retail,Demand estimation...
9     Supply chain management,Offshoring,Outsourcing...
10    Transportation,Multi-period service territory ...
11    Location,Alternative-fuel vehicle,Heuristic al...
12    OR in environment and climate change,Retail lo...
13    OR in marketing,Cooperative advertising,Differ...
14    E-commerce,Omni-channel retailing,Order fulfil...
15    Risk management,Pricing,Mean–variance,Demand f...
16    Uncertain consumer taste,Product positioning,S...
17    Supply chain management,Product returns po

In [78]:
j=0
for j in range(0,len(f['Url_link'])):
# for j in range(0,5):
    try:
#         print(f['Doi_link'].iloc[j])
#         print f['Doi_link'].iloc[j].replace("/abs/", "/pdf/")
        link="https://pubsonline.informs.org/doi/pdf/"+f['Url_link'].iloc[j][16:]
        print j
        print link
        r = requests.get(link, stream = True, headers=header)
        print r
        rep=[", ",","]
#         "/Users/mariiapetryk/Documents/UF/Young/Information security_Informs/"+
        filename=str(f['Author'].iloc[j].replace(", ", "_").replace(",", "_"))+"_"+str(f['Year_publish'].iloc[j])+".pdf"
        print filename
        with open(filename,"wb") as pdf:
            pdf.write(r.content)
    except:
        pass
    sleep(10)

0
https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2017.2837
<Response [200]>
Aseri_Manmohan_Dawande_Milind_Janakiraman_Ganesh_Mookerjee_Vijay_2017.pdf
1
https://pubsonline.informs.org/doi/pdf/10.1287/isre.2016.0651
<Response [200]>
Chen_Ying-Ju_Huang_Ke-Wei_2016.pdf
2
https://pubsonline.informs.org/doi/pdf/10.1287/isre.1120.0453
<Response [200]>
Ghose_Anindya_Goldfarb_Avi_Han_Sang Pil_2012.pdf
3
https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2014.1945
<Response [200]>
Ghose_Anindya_Han_Sang Pil_2014.pdf
4
https://pubsonline.informs.org/doi/pdf/10.1287/isre.2015.0615
<Response [200]>
Hann_Il-Horn_Koh_Byungwan_Niculescu_Marius F._2016.pdf
5
https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2014.1928
<Response [200]>
Ma_Liye_Krishnan_Ramayya_Montgomery_Alan L._2014.pdf
6
https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2015.2270
<Response [200]>
Parker_Chris_Ramdas_Kamalini_Savva_Nicos_2016.pdf
7
https://pubsonline.informs.org/doi/pdf/10.1287/isre.2016.0664
<Response [200]>
